In [2]:
import pandas as pd
from pathlib import Path
import multiprocessing as mp

data_folders = [
    "/media/sagarkumar/New Volume/SAGAR/200/200",
    "/media/sagarkumar/New Volume/SAGAR/200-400/200-400",
    "/media/sagarkumar/New Volume/SAGAR/400-600/400-600",
    "/media/sagarkumar/New Volume/SAGAR/600-759/600-759",
]

# ──────────────────────────────────────────────────────────
# build the flat list of CSV paths
# ──────────────────────────────────────────────────────────
csv_files = [p for folder in data_folders
               for p in Path(folder).glob("*.csv")
               if Path(folder).is_dir()]





In [3]:
import gc

# Create sets to hold unique PARA and VOLTAGE values
unique_para = set()
unique_voltage = set()


for folder in data_folders:
    folder_path = Path(folder)
    for csv_path in folder_path.glob("*.csv"):
        try:
            df = pd.read_csv(csv_path, dtype=str, low_memory=False)

            if "PARA" in df.columns:
                unique_para.update(df["PARA"].dropna().unique())
                print(f"Processed {csv_path} for PARA values.")
            if "VOLTAGE" in df.columns:
                unique_voltage.update(df["VOLTAGE"].dropna().unique())
              
        except Exception as e:
            print(f"Error reading {csv_path}: {e}")
     

print("Unique PARA values:")
print(unique_para)
print("\nUnique VOLTAGE values:")
print(unique_voltage)

Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000182.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000183.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000184.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000185.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000186.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000187.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000188.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000189.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA000000000190.csv for PARA values.
Processed /media/sagarkumar/New Volume/SAGAR/200/200/2025-05-07_SCADA0000

In [2]:
# load the master list of SWNO values
import pandas as pd
df_all = pd.read_csv('/media/sagarkumar/New Volume/SAGAR/all_unique_SWNO.csv', dtype=str)


# load the per-file SWNO list (each row contains semicolon-separated SWNO strings)
df_each = pd.read_csv('/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_withoutDT.csv')
set_feeder = set(df_each["FEEDER_ID"].dropna().unique())

In [13]:
df_each["feeder_match"] = df_all["SWNO"].isin(set_feeder)

print(f"Rows with matching FEEDER_ID: {df_each['feeder_match'].sum()}")

Rows with matching FEEDER_ID: 1091


In [3]:
len(df_each["FEEDER_ID"].unique())

1100

USING PANDAS

In [4]:
import pandas as pd
from pathlib import Path
import concurrent.futures
import os
import re # Import the regular expression module

# List of folders containing the CSV files
data_folders = [
    "/media/sagarkumar/New Volume/SAGAR/200/200",
    "/media/sagarkumar/New Volume/SAGAR/200-400/200-400",
    "/media/sagarkumar/New Volume/SAGAR/400-600/400-600",
    "/media/sagarkumar/New Volume/SAGAR/600-759/600-759",
]

# --- Configuration ---
# The name of the column to look for. The search will be case-insensitive.
SWNO_COLUMN_NAME_TARGET = 'swno'

# Path for the output file
output_file = "/media/sagarkumar/New Volume/SAGAR/unique_swno_faster.csv"


def find_actual_column_name(columns, target_name):
    """Finds the actual column name in a case-insensitive way."""
    for col in columns:
        if col.lower() == target_name.lower():
            return col
    return None

def process_file(file_path):
    """
    Worker function to be run in parallel.
    Reads a single CSV file and returns a set of unique 'swno' values.
    """
    try:
        # First, read only the header to find the correct column name
        df_header = pd.read_csv(file_path, nrows=0, on_bad_lines='skip', engine='c')
        actual_col_name = find_actual_column_name(df_header.columns, SWNO_COLUMN_NAME_TARGET)
        
        if not actual_col_name:
            # Column not found in this file
            return set()
            
        # Now read the correct column from the file
        df = pd.read_csv(file_path, usecols=[actual_col_name], on_bad_lines='skip', engine='c')
        
        # Return a set of unique, non-null values from the column
        return set(df[actual_col_name].dropna().unique())

    except Exception:
        # Suppress errors for files that might be corrupted or unreadable
        return set()


if __name__ == "__main__":
    print("Gathering all CSV file paths...")
    all_files_to_process = []
    for folder_path in data_folders:
        folder = Path(folder_path)
        if folder.is_dir():
            all_files_to_process.extend(list(folder.glob('*.csv')))
        else:
            print(f"Warning: Folder not found at {folder_path}. Skipping.")
    
    if not all_files_to_process:
        print("No CSV files found to process. Exiting.")
    else:
        print(f"Found {len(all_files_to_process)} total files. Starting parallel processing...")
        
        master_unique_swno_set = set()

        with concurrent.futures.ProcessPoolExecutor() as executor:
            results = executor.map(process_file, all_files_to_process)
            for unique_set in results:
                if unique_set: 
                    master_unique_swno_set.update(unique_set)

        print("\nAll files processed. Consolidating and cleaning results...")
        
        # --- DIAGNOSTIC STEP ---
        # Print a small sample of the raw data found before cleaning
        sample_size = min(10, len(master_unique_swno_set))
        if sample_size > 0:
            print(f"Sample of raw unique values found: {list(master_unique_swno_set)[:sample_size]}")
        else:
            print("Warning: No unique values were found in any files. Please check column name and file contents.")
        
        # --- ROBUST CLEANING ---
        # Convert the set to a Series for efficient processing
        s = pd.Series(list(master_unique_swno_set), dtype=str)
        
        # Extract all digits from each string. Handles cases like 'SW-123' or 'abc123def'.
        s = s.str.extract('(\d+)').iloc[:, 0]
        
        # Convert the extracted digits to a numeric type, coercing errors
        s = pd.to_numeric(s, errors='coerce')
        
        # Drop any values that are still null and get the unique integers
        cleaned_values = s.dropna().astype(int).unique()
        
        # Convert the final clean array to a DataFrame
        unique_df = pd.DataFrame(cleaned_values, columns=[SWNO_COLUMN_NAME_TARGET])
        
        # Sort the values for cleaner output
        unique_df = unique_df.sort_values(by=SWNO_COLUMN_NAME_TARGET).reset_index(drop=True)

        # Save the DataFrame to the specified output CSV file
        try:
            unique_df.to_csv(output_file, index=False)
            print(f"\n✅ Success! All unique 'swno' values have been saved to:")
            print(output_file)
            print(f"Total unique values found: {len(unique_df)}")
        except Exception as e:
            print(f"\n❌ Error: Could not save the output file. Reason: {e}")


Gathering all CSV file paths...
Found 756 total files. Starting parallel processing...


/tmp/ipykernel_115329/3502182632.py:45: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=[actual_col_name], on_bad_lines='skip', engine='c')
/tmp/ipykernel_115329/3502182632.py:45: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=[actual_col_name], on_bad_lines='skip', engine='c')
/tmp/ipykernel_115329/3502182632.py:45: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=[actual_col_name], on_bad_lines='skip', engine='c')
/tmp/ipykernel_115329/3502182632.py:45: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, usecols=[actual_col_name], on_bad_lines='skip', engine='c')
/tmp/ipykernel_115329/3502182632.py:45: DtypeWarning: Columns (2) have mixed typ


All files processed. Consolidating and cleaning results...
Sample of raw unique values found: [np.int64(1), '33280', np.int64(11), np.int64(21), '7832', '39941', '33288', '29286', '4427', '29333']

✅ Success! All unique 'swno' values have been saved to:
/media/sagarkumar/New Volume/SAGAR/unique_swno_faster.csv
Total unique values found: 2236


In [17]:
import pandas as pd
df_all = pd.read_csv('/media/sagarkumar/New Volume/SAGAR/unique_swno_from200-759.csv', dtype=str)


# load the per-file SWNO list (each row contains semicolon-separated SWNO strings)
df_each = pd.read_csv('/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_withoutDT.csv')
set_feeder = set(df_each["FEEDER_ID"].dropna().unique())

In [18]:
df_each["feeder_match"] = df_all["SWNO"].isin(set_feeder)

print(f"Rows with matching FEEDER_ID: {df_each['feeder_match'].sum()}")

Rows with matching FEEDER_ID: 0


In [19]:
len(df_all)

286

In [20]:
len(set_feeder)

1100

In [23]:
import pandas as pd

# --- Configuration ---
# Please specify the full paths to your two input files below.
file1_path = "/media/sagarkumar/New Volume/SAGAR/all_unique_SWNO.csv"
file2_path = "/media/sagarkumar/New Volume/SAGAR/unique_swno_from200-759.csv"

# Specify the path for the final combined and sorted output file.
output_file_path = "combined_unique_swno.csv"

# The name of the column you want to combine. This is case-insensitive.
COLUMN_NAME_TARGET = 'SWNO'
# --- End of Configuration ---


def find_actual_column_name(columns, target_name):
    """Finds the actual column name in a case-insensitive way."""
    for col in columns:
        if col.lower() == target_name.lower():
            return col
    return None

def read_swno_from_file(filepath):
    """Reads the target column from a single file, returning a set of values."""
    try:
        print(f"Reading file: {filepath}")
        # Read only the header first to find the correct column name
        header_df = pd.read_csv(filepath, nrows=0, on_bad_lines='skip')
        actual_col_name = find_actual_column_name(header_df.columns, COLUMN_NAME_TARGET)

        if not actual_col_name:
            print(f"  - Warning: Column '{COLUMN_NAME_TARGET}' not found in this file. Skipping.")
            return set()

        # Read the full column using the correct name
        df = pd.read_csv(filepath, usecols=[actual_col_name], on_bad_lines='skip')
        
        # Clean the data: convert to numeric, dropping non-numeric values
        s = pd.to_numeric(df[actual_col_name], errors='coerce')
        cleaned_values = s.dropna()
        
        print(f"  - Found {len(cleaned_values)} valid entries.")
        return set(cleaned_values)

    except FileNotFoundError:
        print(f"  - Error: File not found at {filepath}. Please check the path.")
        return set()
    except Exception as e:
        print(f"  - An unexpected error occurred while reading {filepath}: {e}")
        return set()

# Use a set to automatically handle uniqueness as we combine the files
combined_swno_set = set()

# Read and process the first file
swno_from_file1 = read_swno_from_file(file1_path)
combined_swno_set.update(swno_from_file1)

# Read and process the second file
swno_from_file2 = read_swno_from_file(file2_path)
combined_swno_set.update(swno_from_file2)

print("\nCombining and sorting results...")

if not combined_swno_set:
    print("No valid SWNO data was found in either file. Output file will not be created.")
else:
    # Convert the final set to a DataFrame for sorting and saving
    unique_df = pd.DataFrame(list(combined_swno_set), columns=[COLUMN_NAME_TARGET])
    
    # Ensure the column is integer type for correct sorting
    unique_df[COLUMN_NAME_TARGET] = unique_df[COLUMN_NAME_TARGET].astype(int)
    
    # Sort the values in ascending order
    unique_df = unique_df.sort_values(by=COLUMN_NAME_TARGET).reset_index(drop=True)
    
    # Save the final DataFrame to a new CSV file
    try:
        unique_df.to_csv(output_file_path, index=False)
        print(f"\n✅ Success! Combined unique values have been saved to:")
        print(output_file_path)
        print(f"Total unique values found: {len(unique_df)}")
    except Exception as e:
        print(f"\n❌ Error: Could not save the output file. Reason: {e}")



Reading file: /media/sagarkumar/New Volume/SAGAR/all_unique_SWNO.csv
  - Found 286 valid entries.
Reading file: /media/sagarkumar/New Volume/SAGAR/unique_swno_from200-759.csv
  - Found 2236 valid entries.

Combining and sorting results...

✅ Success! Combined unique values have been saved to:
combined_unique_swno.csv
Total unique values found: 2240


In [27]:
import pandas as pd

# --- CONFIGURATION: PLEASE EDIT THIS SECTION ---

# File 1 Details
file1_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/combined_unique_swno.csv"
file1_column_name = "SWNO"  # e.g., "swno"

# File 2 Details
file2_path = "/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_withoutDT.csv"
file2_column_name = "FEEDER_ID" # e.g., "FEEDER_ID"

# Output File Path
# This file will contain only the values that exist in BOTH files.
output_file_path = "/path/to/your/output/matching_values.csv"

# --- END OF CONFIGURATION ---


def find_actual_column_name(columns, target_name):
    """Helper function to find a column name, ignoring case."""
    for col in columns:
        if str(col).lower() == str(target_name).lower():
            return col
    return None

def get_unique_values_from_file(filepath, column_name):
    """
    Reads a file, extracts unique values from a specific column,
    cleans them, and returns them as a set.
    """
    print(f"Processing file: {filepath}...")
    try:
        # Read just the header to find the correct column name (case-insensitive)
        header_df = pd.read_csv(filepath, nrows=0, on_bad_lines='skip')
        actual_col_name = find_actual_column_name(header_df.columns, column_name)

        if not actual_col_name:
            print(f"  - Error: Column '{column_name}' not found. Please check the column name.")
            return set()

        # Read the full column using the correct name
        df = pd.read_csv(filepath, usecols=[actual_col_name], on_bad_lines='skip')
        
        # --- Data Cleaning ---
        # Convert all values to string, extract digits, and then convert to numbers.
        # This handles mixed data types (e.g., '123' vs 123) and text prefixes (e.g., 'SW-123').
        s = pd.Series(df[actual_col_name].dropna().unique(), dtype=str)
        s = s.str.extract('(\d+)').iloc[:, 0]
        s = pd.to_numeric(s, errors='coerce')
        
        cleaned_values = set(s.dropna().astype(int))
        
        print(f"  - Found {len(cleaned_values)} unique, clean values.")
        return cleaned_values

    except FileNotFoundError:
        print(f"  - Error: File not found. Please check the path: {filepath}")
        return set()
    except Exception as e:
        print(f"  - An unexpected error occurred: {e}")
        return set()

# --- Main Script ---

# Get the unique values from both files
unique_values_from_file1 = get_unique_values_from_file(file1_path, file1_column_name)
unique_values_from_file2 = get_unique_values_from_file(file2_path, file2_column_name)

# --- Comparison and Reporting ---

print("\n--- Comparison Report ---")
if not unique_values_from_file1 or not unique_values_from_file2:
    print("Could not perform comparison because one of the files could not be processed or contained no valid data.")
else:
    # Use set intersection to find the values that exist in both sets
    matching_values = unique_values_from_file1.intersection(unique_values_from_file2)
    
    # Print the final report
    print(f"Unique values in '{file1_path}' (Column: {file1_column_name}): {len(unique_values_from_file1)}")
    print(f"Unique values in '{file2_path}' (Column: {file2_column_name}): {len(unique_values_from_file2)}")
    print("-" * 25)
    print(f"Number of values that matched: {len(matching_values)}")
    print("-" * 25)


Processing file: /media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/combined_unique_swno.csv...
  - Found 2240 unique, clean values.
Processing file: /media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_withoutDT.csv...
  - Found 1100 unique, clean values.

--- Comparison Report ---
Unique values in '/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/combined_unique_swno.csv' (Column: SWNO): 2240
Unique values in '/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_withoutDT.csv' (Column: FEEDER_ID): 1100
-------------------------
Number of values that matched: 1091
-------------------------


In [28]:
df_new = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_withoutDT.csv")

In [29]:
col = df_new['FEEDER_ID'].unique()

In [30]:
len(col)

1100

In [32]:
df_new_11 = pd.read_csv("/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_full.csv")

In [33]:
col = df_new_11['FEEDER_ID'].unique()

In [34]:
len(col)

1401

In [37]:
import csv
import re
import os

def find_and_save_mismatched_rows(input_file_path, output_file_path, column_name='FROM_TO'):
    """
    Reads a CSV file, finds rows where a specific column does not
    match the 'xxxx-yyyy' format, and saves them to a new CSV file.

    Args:
        input_file_path (str): The path to the input CSV file.
        output_file_path (str): The path to the output CSV file.
        column_name (str): The name of the column to check.
    """
    # This regular expression pattern checks for a string that starts and
    # ends with exactly four digits separated by a single hyphen.
    pattern = re.compile(r'^\d{4}-\d{4}$')

    mismatched_rows = []

    try:
        with open(input_file_path, mode='r', newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)

            # Read the header row to find the index of the target column
            header = next(reader)
            if column_name not in header:
                print(f"Error: Column '{column_name}' not found in the CSV file.")
                return

            from_to_index = header.index(column_name)
            
            # Prepare the header for the output file, adding a new column for the line number
            output_header = ['Original_Line'] + header
            mismatched_rows.append(output_header)

            # Iterate over each row in the CSV file
            for i, row in enumerate(reader):
                # Using i + 2 because line numbers are 1-based and we skipped the header
                line_number = i + 2
                
                # Ensure the row has enough columns to prevent index errors
                if len(row) > from_to_index:
                    cell_value = row[from_to_index].strip() # Get value and remove whitespace

                    # Check if the cell value does NOT match the pattern
                    if not pattern.match(cell_value):
                        # Prepend the original row number for easy reference
                        row_with_line_num = [line_number] + row
                        mismatched_rows.append(row_with_line_num)
                else:
                    # Handle rows that are shorter than the header
                    row_with_line_num = [f"{line_number} (Malformed Row)"] + row
                    mismatched_rows.append(row_with_line_num)

    except FileNotFoundError:
        print(f"Error: The file '{input_file_path}' was not found.")
        return
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return

    # Save the results to the output file
    if len(mismatched_rows) > 1: # More than just the header
        try:
            with open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
                writer = csv.writer(outfile)
                writer.writerows(mismatched_rows)
            print(f"Found {len(mismatched_rows) - 1} mismatched rows. Results saved to '{output_file_path}'.")
        except Exception as e:
            print(f"An error occurred while writing to the output file: {e}")
    else:
        print(f"All rows in the '{column_name}' column are correctly formatted. No output file was created.")

# 2. Specify the path to your CSV file.
#    Replace 'sample_data.csv' with the actual path to your file.
input_csv = '/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_full.csv'
output_csv = 'mismatched_rows.csv'

# 3. Run the function.
find_and_save_mismatched_rows(input_csv, output_csv)



Found 16540 mismatched rows. Results saved to 'mismatched_rows.csv'.


In [39]:
import csv
import re
import os

def find_and_save_matching_rows(input_file_path, output_file_path, column_name='FROM_TO'):
    """
    Reads a CSV file, finds rows where the 'FROM_TO' column matches
    a 'xxxx-A...' format (four digits, a hyphen, then an alphabet character),
    and saves them to a new CSV file.

    Args:
        input_file_path (str): The path to the input CSV file.
        output_file_path (str): The path to the output CSV file.
        column_name (str): The name of the column to check.
    """
    # This regular expression checks for a string that starts with four digits,
    # followed by a hyphen, and then followed by an alphabet character.
    # ^\d{4}   - starts with exactly four digits
    # -        - followed by a literal hyphen
    # [a-zA-Z] - followed by any single uppercase or lowercase letter
    # .* - matches any character (except for line terminators) afterward
    pattern = re.compile(r'^\d{4}-[a-zA-Z].*')

    matching_rows = []

    try:
        with open(input_file_path, mode='r', newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)

            # Read the header row to find the index of the target column
            header = next(reader)
            if column_name not in header:
                print(f"Error: Column '{column_name}' not found in the CSV file.")
                return

            from_to_index = header.index(column_name)
            
            # Prepare the header for the output file, including a column for the original line number
            output_header = ['Original_Line'] + header
            matching_rows.append(output_header)

            # Iterate over each row in the CSV file
            for i, row in enumerate(reader):
                line_number = i + 2  # Line numbers are 1-based, and we skipped the header
                
                if len(row) > from_to_index:
                    cell_value = row[from_to_index].strip()

                    # Check if the cell value MATCHES the specified pattern
                    if pattern.match(cell_value):
                        # Prepend the original line number and add it to our list
                        row_with_line_num = [line_number] + row
                        matching_rows.append(row_with_line_num)

    except FileNotFoundError:
        print(f"Error: The file '{input_file_path}' was not found.")
        return
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return

    # Save the matching rows to the output file
    if len(matching_rows) > 1:  # Check if any matches were found besides the header
        try:
            with open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
                writer = csv.writer(outfile)
                writer.writerows(matching_rows)
            print(f"Found {len(matching_rows) - 1} matching rows. Results saved to '{output_file_path}'.")
        except Exception as e:
            print(f"An error occurred while writing to the output file: {e}")
    else:
        print(f"No rows matching the pattern were found. No output file was created.")


# --- How to use this script --

# 2. Specify the path to your input and output CSV files.
input_csv = '/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_full.csv'
output_csv = 'mismatched_rows.csv'

# 3. Run the function.
find_and_save_mismatched_rows(input_csv, output_csv)


Found 16540 mismatched rows. Results saved to 'mismatched_rows.csv'.


In [48]:
import pandas as pd
import re

# Function to check if a string is strictly digit-digit (e.g., 123-456)
def from_to_is_numeric(s):
    return bool(re.fullmatch(r'\d+-\d+', str(s)))

# Path to your input and output CSV files
input_csv = '/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/final_two_column_with_rank_11_full.csv'
output_csv = 'from_to_not_numeric_rows.csv'

# Read the CSV into a DataFrame
merged = pd.read_csv(input_csv)

# Remove rows where FROM_TO is digit-digit
filtered = merged[~merged['FROM_TO'].apply(from_to_is_numeric)]

# Save the result to a new CSV
filtered.to_csv(output_csv, index=False)

print(f"Rows where FROM_TO is digit-digit are removed. Output saved to '{output_csv}'.")


Rows where FROM_TO is digit-digit are removed. Output saved to 'from_to_not_numeric_rows.csv'.


In [50]:
df_n = pd.read_csv('/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/from_to_not_numeric_rows.csv')

In [56]:
import pandas as pd

# Load your DataFrame
df_n = pd.read_csv('/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/from_to_not_numeric_rows.csv')

# --- Replace 'column_A' and 'column_B' with your actual column names ---
column1_name = 'SOURCE_LOCATION'
column2_name = 'DESTINATION_LOCATION'

# Count the number of rows where the values in the two columns are the same
same_value_count = (df_n[column1_name] == df_n[column2_name]).sum()

print(f"The number of rows where '{column1_name}' and '{column2_name}' have the same value is: {same_value_count}")

The number of rows where 'SOURCE_LOCATION' and 'DESTINATION_LOCATION' have the same value is: 7196


In [60]:
import pandas as pd

# Define your input file path
input_file = '/media/sagarkumar/New Volume/SAGAR/IPYNB_FILE/from_to_not_numeric_rows.csv'
# Define where you want to save the output file
output_file = 'non_matching_rows.csv'


try:
    # Load your DataFrame
    df = pd.read_csv(input_file)

    # --- Replace 'column_A' and 'column_B' with your actual column names ---
    column1_name = 'SOURCE_LOCATION'
    column2_name = 'DESTINATION_LOCATION'

    # Check if the columns exist in the DataFrame
    if column1_name not in df.columns or column2_name not in df.columns:
        print(f"Error: One or both columns ('{column1_name}', '{column2_name}') not found in the file.")
        print(f"Available columns are: {list(df.columns)}")
    else:
        # Find the rows that do NOT match
        # The `!=` operator checks for inequality
        non_matching_rows_df = df[df[column1_name] != df[column2_name]]

        # Save the resulting DataFrame to a new CSV file
        # index=False prevents pandas from writing the DataFrame index as a column
        non_matching_rows_df.to_csv(output_file, index=False)

        print(f"Found {len(non_matching_rows_df)} non-matching rows.")
        print(f"These rows have been saved to '{output_file}'.")

except FileNotFoundError:
    print(f"Error: The file was not found at '{input_file}'")
except Exception as e:
    print(f"An error occurred: {e}")

Found 652 non-matching rows.
These rows have been saved to 'non_matching_rows.csv'.
